In [1]:
# Initial Imports

import pandas as pd
import numpy as np
import requests
import json
import datetime as dt
from pathlib import Path
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Import flair pre-trained sentiment model
from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')
# Import flair Sentence to process input text
from flair.data import Sentence
import regex as re
from sentiment_functions import clean_text, score_flair
import datetime
# BDay is business day, not birthday...
from pandas.tseries.offsets import BDay




2022-01-18 20:28:42,223 loading file C:\Users\abhis\.flair\models\sentiment-en-mix-distillbert_4.pt
2022-01-18 20:28:46,190 loading file C:\Users\abhis\.flair\models\sentiment-en-mix-distillbert_4.pt


In [3]:
# Taking user input for number of days to retrieve data for, then creating the date range to iterate through for the API call


def getTwitterData():
        
    tweets_df = pd.DataFrame()
    numDays = int(input('How many days would you like to retrieve twitter data for?'))
    hashtag = input('What hashtag would you like to retrieve data for?')
    
    today = datetime.datetime.today()
    start = today - pd.Timedelta(days = numDays)
    
    

    #Since we are retrieving data we want to correlate with markets, we are using 'bdate_range' to only return business days                
    date_range = pd.bdate_range(end = dt.date.today(), periods = int(numDays))
        
    #Iterating through the date_range with an API call for each day to maximize data returned 
    # @TODO -- use function to limit API calls to not exceed limits
    for i in range(0, len(date_range) -1, 1):
    
        start_date = date_range[i]
        end_date = date_range[i+1] 
    
    
        start_date = pd.Timestamp.isoformat(start_date)
        end_date = pd.Timestamp.isoformat(end_date)
    
    
# Making the API call to retrieve tweets 

        source = "https://twitter32.p.rapidapi.com/getSearch"

# These parameters need to be set before running API call... could in the future receive inputs from user
## TODO -- if input is received, datetime will need to be formatted correctly for API call

        hashtag = hashtag

        querystring = {"hashtag": hashtag, "start_date": start_date,"lang":"en"}
        headers = {
        'x-rapidapi-host': "twitter32.p.rapidapi.com",
        'x-rapidapi-key': "49e5cedc9fmsh6a2df83dacfc4c1p1c3469jsn3edb2e8cb9df"
        }
   
        
        response = requests.get(source, headers=headers, params=querystring).json()
        df = pd.DataFrame(response['data']['tweets']).T
        tweets_df = pd.concat([df, tweets_df])
        display(start_date, end_date)
    
    return tweets_df

tweets_df = getTwitterData()
display(tweets_df)


        


How many days would you like to retrieve twitter data for? 5
What hashtag would you like to retrieve data for? tsla


'2022-01-12T00:00:00'

'2022-01-13T00:00:00'

'2022-01-13T00:00:00'

'2022-01-14T00:00:00'

'2022-01-14T00:00:00'

'2022-01-17T00:00:00'

'2022-01-17T00:00:00'

'2022-01-18T00:00:00'

created_at                   id  \
1483529785003458570  Tue Jan 18 20:00:38 +0000 2022  1483529785003458570   
1483536069450739713  Tue Jan 18 20:25:36 +0000 2022  1483536069450739713   
1483633282482847744  Wed Jan 19 02:51:53 +0000 2022  1483633282482847744   
1483604425902071816  Wed Jan 19 00:57:13 +0000 2022  1483604425902071816   
1483502134259851269  Tue Jan 18 18:10:45 +0000 2022  1483502134259851269   
...                                             ...                  ...   
1483513569681584128  Tue Jan 18 18:56:12 +0000 2022  1483513569681584128   
1483603512873345025  Wed Jan 19 00:53:36 +0000 2022  1483603512873345025   
1483467626684862469  Tue Jan 18 15:53:38 +0000 2022  1483467626684862469   
1483468006869065731  Tue Jan 18 15:55:08 +0000 2022  1483468006869065731   
1483438765595148297  Tue Jan 18 13:58:57 +0000 2022  1483438765595148297   

                                  id_str  \
1483529785003458570  1483529785003458570   
1483536069450739713  1483536069450739713   
1483633282482847744  1483633282482847744   
1483604425902071816  1483604425902071816   
1483502134259851269  1483502134259851269   
...                                  ...   
1483513569681584128  1483513569681584128   
1483603512873345025  1483603512873345025   
1483467626684862469  1483467626684862469   
1483468006869065731  1483468006869065731   
1483438765595148297  1483438765595148297   

                                                             full_text  \
1483529785003458570  Cash isn’t trash when growth stocks are fallin...   
1483536069450739713  #TSLA is a good loss because it's small and in...   
1483633282482847744                             Believe in the future!   
1483604425902071816  tomorrow my 👀 on $TSLA #TSLA weekly golden arc...   
1483502134259851269  Waiting for #TSLA to 🚀 🔥!!!!!!! https://t.co/0...   
...                                                                ...   
1483513569681584128      #TSLA is Innovator’s Dilemma in slow motion..   
1483603512873345025  $TSLA Below $1,030 brings 💥 #TSLA https://t.co...   
1483467626684862469  Here is heikin ashi 1M chart for #TSLA . Candl...   
1483468006869065731  Ok at gym when im done i expect #TSLA to hit 1...   
1483438765595148297  Me waiting for the markets to open #TSLA $TSLA...   

                    truncated display_text_range  \
1483529785003458570     False            [0, 52]   
1483536069450739713     False            [0, 63]   
1483633282482847744     False            [0, 22]   
1483604425902071816     False            [0, 46]   
1483502134259851269     False            [0, 31]   
...                       ...                ...   
1483513569681584128     False            [0, 45]   
1483603512873345025     False            [0, 33]   
1483467626684862469     False            [0, 69]   
1483468006869065731     False            [0, 80]   
1483438765595148297     False            [0, 46]   

                                                              entities  \
1483529785003458570  {'hashtags': [], 'symbols': [], 'user_mentions...   
1483536069450739713  {'hashtags': [{'text': 'TSLA', 'indices': [0, ...   
1483633282482847744  {'hashtags': [], 'symbols': [], 'user_mentions...   
1483604425902071816  {'hashtags': [{'text': 'TSLA', 'indices': [23,...   
1483502134259851269  {'hashtags': [{'text': 'TSLA', 'indices': [12,...   
...                                                                ...   
1483513569681584128  {'hashtags': [{'text': 'TSLA', 'indices': [0, ...   
1483603512873345025  {'hashtags': [{'text': 'TSLA', 'indices': [28,...   
1483467626684862469  {'hashtags': [{'text': 'TSLA', 'indices': [33,...   
1483468006869065731  {'hashtags': [{'text': 'TSLA', 'indices': [32,...   
1483438765595148297  {'hashtags': [{'text': 'TSLA', 'indices': [35,...   

                                                                source  \
1483529785003458570  <a href="http://twitter.com/#!/download/ipad" ...   
1483536069450739713  <a href="https://mobile.twi

In [ ]:
'''def getTodayTweets(current_df_path)
    
    todaysTweets_df = pd.DataFrame()
    
    current_df_path = Path('./twitter_flair.csv'
    currentTweets_df = pd.read_csv(current_df_path, parse_dates = True, infer_datetime_format=True, index_col='Date')
    hashtag = input('Please input desired hashtag to confirm')
    
    end_date = datetime.datetime.today()
    start_date = today - pd.Timedelta(days=1)
    
    
    #Iterating through the date_range with an API call for each day to maximize data returned 
    # @TODO -- use function to limit API calls to not exceed limit
    
    start_date = pd.Timestamp.isoformat(start_date)
    end_date = pd.Timestamp.isoformat(end_date)
    
    
    # Making the API call to retrieve tweets 

    source = "https://twitter32.p.rapidapi.com/getSearch"

    # These parameters need to be set before running API call... could in the future receive inputs from user
    ## TODO -- if input is received, datetime will need to be formatted correctly for API call

    hashtag = hashtag
    querystring = {"hashtag": hashtag, "start_date": start_date,"end_date": end_date,"lang":"en"}
    headers = {
        'x-rapidapi-host': "twitter32.p.rapidapi.com",
        'x-rapidapi-key': "49e5cedc9fmsh6a2df83dacfc4c1p1c3469jsn3edb2e8cb9df"
        }
   
        
    response = requests.get(source, headers=headers, params=querystring).json()
    todaysTweets_df = pd.DataFrame(response['data']['tweets']).T
    updatedTweets_df = pd.concat([todays_Tweets_df, currentTweets])
    display(start_date, end_date)
                           
    
    return updatedTweets_df

updatedTweets_df = getTwitterData('./twitter_flair.csv')
display(tweets_df)'''

In [4]:
# Parsing and formatting the dataframe to index by date created and isolating the 'full_text' and 'retweet_count' columns for sentiment analysis
df_filtered = tweets_df.loc[:,["created_at","full_text", 'retweet_count']]
df_filtered.index = pd.to_datetime(df_filtered['created_at'])
df_filtered.index = df_filtered.index.date
df_filtered.index.name = 'Date'

In [5]:
df_filtered.drop(columns='created_at', inplace=True)
df_filtered

full_text retweet_count
Date                                                                       
2022-01-18  Cash isn’t trash when growth stocks are fallin...            15
2022-01-18  #TSLA is a good loss because it's small and in...             0
2022-01-19                             Believe in the future!          7444
2022-01-19  tomorrow my 👀 on $TSLA #TSLA weekly golden arc...             0
2022-01-18  Waiting for #TSLA to 🚀 🔥!!!!!!! https://t.co/0...             0
...                                                       ...           ...
2022-01-18      #TSLA is Innovator’s Dilemma in slow motion..             3
2022-01-19  $TSLA Below $1,030 brings 💥 #TSLA https://t.co...             1
2022-01-18  Here is heikin ashi 1M chart for #TSLA . Candl...             0
2022-01-18  Ok at gym when im done i expect #TSLA to hit 1...             0
2022-01-18  Me waiting for the markets to open #TSLA $TSLA...             1

[85 rows x 2 columns]

In [6]:
# We need to clean up our tweets so they can be analyzed in string format
# Applying our imported 'clean_text' function to text column of our dataframe
df_filtered["full_text"] = df_filtered["full_text"].apply(clean_text)
#df_filtered = df_filtered.sort_index()
df_filtered

full_text retweet_count
Date                                                                       
2022-01-18  Cash isn t trash when growth stocks are fallin...            15
2022-01-18  TSLA is a good loss because it s small and in ...             0
2022-01-19                              Believe in the future          7444
2022-01-19         tomorrow my on TSLA TSLA weekly golden arc             0
2022-01-18                                Waiting for TSLA to             0
...                                                       ...           ...
2022-01-18         TSLA is Innovator s Dilemma in slow motion             3
2022-01-19                       TSLA Below 1 030 brings TSLA             1
2022-01-18  Here is heikin ashi 1M chart for TSLA Candles ...             0
2022-01-18  Ok at gym when im done i expect TSLA to hit 11...             0
2022-01-18       Me waiting for the markets to open TSLA TSLA             1

[85 rows x 2 columns]

In [7]:
df_filtered = df_filtered.reset_index()
tweet_dic = {}
for i in range(len(df_filtered)):
    date = df_filtered['Date'][i]
    summary = df_filtered['full_text'][i]
    if date in tweet_dic:
        tweet_dic[date].append(summary)
    else:
        tweet_dic[date] = [summary]
        
twitter_df = pd.DataFrame.from_dict(tweet_dic, orient = 'index')
twitter_df = twitter_df.reset_index()
twitter_df['index'] = pd.to_datetime(twitter_df['index'])

In [8]:
df_filtered

Date                                          full_text  \
0   2022-01-18  Cash isn t trash when growth stocks are fallin...   
1   2022-01-18  TSLA is a good loss because it s small and in ...   
2   2022-01-19                              Believe in the future   
3   2022-01-19         tomorrow my on TSLA TSLA weekly golden arc   
4   2022-01-18                                Waiting for TSLA to   
..         ...                                                ...   
80  2022-01-18         TSLA is Innovator s Dilemma in slow motion   
81  2022-01-19                       TSLA Below 1 030 brings TSLA   
82  2022-01-18  Here is heikin ashi 1M chart for TSLA Candles ...   
83  2022-01-18  Ok at gym when im done i expect TSLA to hit 11...   
84  2022-01-18       Me waiting for the markets to open TSLA TSLA   

   retweet_count  
0             15  
1              0  
2           7444  
3              0  
4              0  
..           ...  
80             3  
81             1  
82             0  
83             0  
84             1  

[85 rows x 3 columns]

In [9]:
twitter_df

index                                                  0  \
0 2022-01-18  Cash isn t trash when growth stocks are fallin...   
1 2022-01-19                              Believe in the future   
2 2022-01-17                          History often rhymes TSLA   
3 2022-01-14  Government says Tesla is trying to pressurise ...   
4 2022-01-16  I LOVE TSLA I have decided 2 day trade NOT buy...   
5 2022-01-15  Price of dodgecoin increased yesterday on anno...   
6 2022-01-13  TSLA TSLA Nice breakout of the falling wedge f...   
7 2022-01-12  Redditors on r WhitePeopleTwitter still think ...   

                                                   1  \
0  TSLA is a good loss because it s small and in ...   
1         tomorrow my on TSLA TSLA weekly golden arc   
2                          History often rhymes TSLA   
3  I love how all automakers are starting to go a...   
4  TSLA If breaks the immediate resistance level ...   
5                                               None   
6  TSLA TSLA All time highs long term Buy Tesla i...   
7  The name change is official now Tesla Giga Tex...   

                                                   2  \
0                                Waiting for TSLA to   
1  How when young investors are putting money in ...   
2  Honestly if tsla autopilot cuts deaths in half...   
3  Government says Tesla is trying to pressurise ...   
4                                               None   
5                                               None   
6                                               None   
7                                               None   

                                                   3  \
0  Sold my tsla shares and bought Dwac instead Th...   
1                             TSLA more to come TSLA   
2  Punjab Congress chief Navjot Singh Sidhu tweet...   
3                                               None   
4                                               None   
5                                               None   
6                                               None   
7                                               None   

                                                   4  \
0                TSLA weekly 1020 puts bouta run ITM   
1  Following were the Top 5 tickers mentioned on ...   
2  My workaround until I take delivery of the Cyb...   
3                                               None   
4                                               None   
5                                               None   
6                                               None   
7                                               None   

                                                   5  \
0               The trend is your friend Enough TSLA   
1  I didn t learn this Although I don t regret bu...   
2  TSLA formed a pennant on the 15m last trading ...   
3                                               None   
4                                               None   
5                                               None   
6                                               None   
7                                               None   

                                                   6  \
0  Tesla gaining on BMW for U S luxury sales crow...   
1                       TSLA Below 1 030 brings TSLA   
2                          History often rhymes TSLA   
3                                               None   
4                                               None   
5                                               None   
6                                               None   
7                                               None   

                                                   7  \
0             Credit Suisse TSLA PT UP 1025 from 830   
1                              Believe in the future   
2  I request to gift me few TSLA stocks as I miss...   
3                                               None   
4                                               None   
5                                    

In [10]:


# Get sentiment score for each review
df_filtered['scores_flair'] = df_filtered['full_text'].apply(lambda s: score_flair(s)[0])
# Predict sentiment label for each review
df_filtered['pred_flair'] = df_filtered['full_text'].apply(lambda s: score_flair(s)[1])
# Check the distribution of the score
df_filtered['scores_flair'].describe()
# Change the label of flair prediction to 0 if negative and 1 if positive
#mapping = {'NEGATIVE': 0, 'POSITIVE': 1}
#df_filtered['pred_flair_numerical'] = df_filtered['pred_flair'].map(mapping)
df_filtered['pred_flair'].describe()


count           85
unique           2
top       POSITIVE
freq            47
Name: pred_flair, dtype: object

In [11]:
#Converting the categorical variable columns into numerical with get_dummies
df_dummies = pd.get_dummies(df_filtered['pred_flair'])
# Dropping the row if the scores_flair confidence level is below 0.75
df_filtered.drop(df_filtered[df_filtered.scores_flair < 0.75].index, inplace=True)
# Creating the 'df_signals' dataframe that we will use to append to our primary dataset, using only the columns with relevant numerical values, resampled by day
df_signals = df_filtered.drop(columns=['scores_flair', 'pred_flair'])
final_df = pd.concat([df_signals, df_dummies]).resample('D').sum()
display(final_df)  
#final_df.to_csv('twitter_flair.csv')

    
    


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'

### After pulling tweet data, performing sentiment analysis using flair and then filtered and resampling the dataframe, we return a dataframe with datetime indices with two columns -- flair_negative, which is the sum of all negative tweets for the day, and flair_positive, which is the sum of all positive tweets for the day -- at some point we could think about factoring in number of retweets or something along those lines to weigh the tweets by their impact, but the results from the API call already filter by this metric, so could be superfluous.